# Projet 4

In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

    

In [3]:
def get_prices_from_url(url):
    results = {}
    r = requests.get(url)
    #print(r.content)
    soup = BeautifulSoup(r.content)
    
    pricing_header = soup.find_all('div', attrs={'class': 'pricing-table-header'})
    pricing_list = soup.find_all('ul', attrs={'class': 'pricing-table-list'})
    
    for i in range(len(pricing_header)):
        cat = pricing_header[i].h2.text
        price = pricing_header[i].find(string=re.compile("\$\d+")).strip()
        storage_line = pricing_list[i].find(string=re.compile("storage"))
        storage = re.match("\d+.B", storage_line)[0]
        db_line = pricing_list[i].find(string=re.compile("database"))
        db = int(re.match("\d+", db_line)[0])
        results[cat] = {'price': price, 'storage': storage,
                            'databases': db}
        
    return results
    
    

get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

In [4]:
get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [5]:
def extract_beer_infos(url):
    infos = {
        'name': None,
        'note': None,
        'price': None,
        'volume': None,
    }
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    name = soup.find("h1").text.strip()
    
    note = int(soup.find(class_="stars")["data-percent"])
    
    price_line = soup.find_all(string=re.compile("(\d+,\d+)\s€"))[0]
    price = float(price_line.replace(',', '.').replace(" €", ''))
    
    volume_line = soup.find(class_="js-beer-volume").text
    volume = int(re.match("\d+", volume_line)[0])
    
    infos['name'] = name
    infos['note'] = note
    infos['price'] = price
    infos['volume'] = volume
    
    return infos


extract_beer_infos("https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33")

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [6]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [7]:
%%time
import multiprocessing
import os

## il y a peut-être plus simple mais... c'est rapide
def go_fast(urls):
    details=[]
    def get_details(t):
        details.append(t)
    
    with multiprocessing.Pool(os.cpu_count()) as p:
        multiple_results = [p.apply_async(worker, [url], callback = get_details) for url in urls]
        [res.get(timeout=1000) for res in multiple_results]
    print("This was damn fast!")
    return details

def worker(url):
    t = extract_beer_infos(url)
    return t

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    r = requests.get(url)
    b = json.loads(r.content)['items']
    
    # Collecter les pages de bières à partir du JSON
    urls = ["https://www.beerwulf.com" + b[i]['contentReference'] for i in range(len(b))]

    # Sequential version (slow):
    #beers = [extract_beer_infos(url) for url in urls]

    # Parallel version (faster):
    beers = go_fast(urls)
    
    return beers

extract_beer_list_infos(URL_BEERLIST_FRANCE)

This was damn fast!
CPU times: user 106 ms, sys: 24.6 ms, total: 131 ms
Wall time: 794 ms


[{'name': 'La Cristal IPA du Mont Blanc',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
 {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 60,
  'price': 38.99,
  'volume': 25},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 70,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
 {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33}]

In [8]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [9]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... 

This was damn fast!


ok

----------------------------------------------------------------------
Ran 3 tests in 1.159s

OK
